In [64]:
from cabocha.analyzer import CaboChaAnalyzer
import pprint
text_path = 'data/chap4/neko.txt'
cabocha_path = 'data/chap5/neko.txt.cabocha'

In [2]:
!cabocha -f1 < $text_path > $cabocha_path

### 40. 係り受け解析結果の読み込み（形態素）

In [3]:
class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1

In [4]:
def parseCabocha(block):
    lst = []
    for line in block.split('\n'):
        if line == '':
            return lst
        elif line[0] == '*':
            continue
        (surface, attr) = line.split('\t')
        attr = attr.split(',')
        lst.append(Morph(surface, attr[6], attr[0], attr[1]))

In [5]:
with open(cabocha_path, mode='rt', encoding='utf-8') as f:
    blockList = f.read().split('EOS\n')
blockList = list(filter(lambda x: x != '', blockList))
blockList = [parseCabocha(block) for block in blockList]
for m in blockList[2]:
    print(vars(m))

{'surface': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'}
{'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'}
{'surface': 'まだ', 'base': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続'}
{'surface': '無い', 'base': '無い', 'pos': '形容詞', 'pos1': '自立'}
{'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}


### 41. 係り受け解析結果の読み込み（文節・係り受け）

In [6]:
class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []

In [7]:
def parseCabocha(block):
    def checkCreateChunk(tmp):
        if len(tmp) > 0:
            c = Chunk(tmp, dst)
            lst.append(c)
            tmp = []
        return tmp
    
    
    lst = []
    tmp = []
    dst = None
    for line in block.split('\n'):
        if line == '':
            tmp = checkCreateChunk(tmp)
        elif line[0] == '*':
            tmp = checkCreateChunk(tmp)
            dst = line.split(' ')[2].rstrip('D')
        else:
            (surface, attr) = line.split('\t')
            attr = attr.split(',')
            tmp.append(Morph(surface, attr[6], attr[0], attr[1]))
    for i, v in enumerate(lst):
        # 係り先なし
        if int(v.dst) == -1:
            continue
        lst[int(v.dst)].srcs.append(i)
    return lst

In [8]:
with open(cabocha_path, mode='rt', encoding='utf-8') as f:
    blockList = f.read().split('EOS\n')
blockList = list(filter(lambda x: x != '', blockList))
blockList = [parseCabocha(block) for block in blockList]
for m in blockList[7]:
    print([mo.surface for mo in m.morphs], m.dst, m.srcs)

['この'] 1 []
['書生', 'という', 'の', 'は'] 7 [0]
['時々'] 4 []
['我々', 'を'] 4 []
['捕え', 'て'] 5 [2, 3]
['煮', 'て'] 6 [4]
['食う', 'という'] 7 [5]
['話', 'で', 'ある', '。'] -1 [1, 6]


### 42. 係り元と係り先の文節の表示

In [9]:
def print_sentence(block):
    s = ''
    for m in block:
        for mo in m.morphs:
            s += mo.surface
    print(s)

In [10]:
for block in blockList[:10]:
    print_sentence(block)
    for m in block:
        if int(m.dst) > -1:
            s = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
            t = ''.join([mo.surface for mo in block[int(m.dst)].morphs if mo.pos != '記号'])
            print(f'{s}\t{t}')

一
　吾輩は猫である。
	猫である
吾輩は	猫である
名前はまだ無い。
名前は	無い
まだ	無い
　どこで生れたかとんと見当がつかぬ。
どこで	生れたか
生れたか	つかぬ
とんと	つかぬ
見当が	つかぬ
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
何でも	薄暗い
薄暗い	所で
じめじめした	所で
所で	泣いて
ニャーニャー	泣いて
泣いて	記憶している
いた事だけは	記憶している
吾輩はここで始めて人間というものを見た。
吾輩は	見た
ここで	始めて
始めて	人間という
人間という	ものを
ものを	見た
しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。
しかも	種族であったそうだ
あとで	聞くと
聞くと	種族であったそうだ
それは	種族であったそうだ
書生という	人間中で
人間中で	種族であったそうだ
一番	獰悪な
獰悪な	種族であったそうだ
この書生というのは時々我々を捕えて煮て食うという話である。
この	書生というのは
書生というのは	話である
時々	捕えて
我々を	捕えて
捕えて	煮て
煮て	食うという
食うという	話である
しかしその当時は何という考もなかったから別段恐しいとも思わなかった。
しかし	思わなかった
その	当時は
当時は	なかったから
何という	考も
考も	なかったから
なかったから	思わなかった
別段	恐し
恐し	思わなかった
いとも	思わなかった
ただ彼の掌に載せられてスーと持ち上げられた時何だかフワフワした感じがあったばかりである。
ただ	載せられて
彼の	掌に
掌に	載せられて
載せられて	持ち上げられた
スーと	持ち上げられた
持ち上げられた	時
時	フワフワした
何だか	フワフワした
フワフワした	感じが
感じが	あったばかりである


### 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [11]:
for block in blockList[:10]:
    for m in block:
        if int(m.dst) > -1:
            if any([mo.pos == '名詞' for mo in m.morphs]) and any([mo.pos == '動詞' for mo in block[int(m.dst)].morphs]):
                s = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
                t = ''.join([mo.surface for mo in block[int(m.dst)].morphs if mo.pos != '記号'])
                print(f'{s}\t{t}')

どこで	生れたか
見当が	つかぬ
所で	泣いて
ニャーニャー	泣いて
いた事だけは	記憶している
吾輩は	見た
ここで	始めて
ものを	見た
あとで	聞くと
我々を	捕えて
掌に	載せられて
スーと	持ち上げられた
時	フワフワした
感じが	あったばかりである


### 44. 係り受け木の可視化

In [12]:
!pip install pydot

In [13]:
import pydot

block = blockList[7]
pairs = []
for m in block:
    if int(m.dst) > -1:
        s = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
        t = ''.join([mo.surface for mo in block[int(m.dst)].morphs if mo.pos != '記号'])
        pairs.append([s, t])
g = pydot.graph_from_edges(pairs)
print(pairs)
g.write_png('data/chap5/graph.png', prog='dot')

[['この', '書生というのは'], ['書生というのは', '話である'], ['時々', '捕えて'], ['我々を', '捕えて'], ['捕えて', '煮て'], ['煮て', '食うという'], ['食うという', '話である']]


### 45. 動詞の格パターンの抽出

In [197]:
case_path = 'data/chap5/neko_case.txt'
case = []
for block in blockList:
    for m in block:
        post = []
        if len(m.srcs) > 0:
            # 動詞が含まれる節
            if any([mo.pos == '動詞' for mo in m.morphs]):
                verb = [mo.base for mo in m.morphs if mo.pos == '動詞']
                
                for src in m.srcs:
                    # 助詞
                    post += [mo.base for mo in block[src].morphs if mo.pos == '助詞']
                if post :
                    post.sort()
                    case.append(f"{verb[0]}\t{' '.join(post)}")
                
with open (case_path, mode='w') as f:
    f.write('\n'.join(case))

In [198]:
!sort $case_path | uniq -c | sort -n -r | head

 565 云う	と
 442 する	を
 249 思う	と
 199 ある	が
 189 なる	に
 174 する	に
 173 見る	て
 127 する	と
 117 する	が
 105 する	に を
sort: Broken pipe


In [199]:
!grep "^する\t" $case_path | sort | uniq -c | sort -n -r | head

 442 する	を
 174 する	に
 127 する	と
 117 する	が
 105 する	に を
  86 する	て を
  59 する	は
  58 する	て
  57 する	が を
  48 する	から


In [60]:
!grep "^見る\t" $case_path | sort | uniq -c | sort -n -r | head

 173 見る	て
  94 見る	を
  21 見る	て て
  20 見る	から
  16 見る	て を
  14 見る	と
  12 見る	で
  11 見る	から て
  11 見る	は て
   8 見る	に


In [61]:
!grep "^与える\t" $case_path | sort | uniq -c | sort -n -r | head

   3 与える	に を
   1 与える	けれども に は を
   1 与える	じゃあ か と は て を
   1 与える	として を か
   1 与える	たり て に を
   1 与える	で だけ に を
   1 与える	に は に対して のみ は も
   1 与える	て が は は と て に を
   1 与える	は て に を に
   1 与える	は て に を


### 46. 動詞の格フレーム情報の抽出

In [148]:
case = []
for block in blockList:
    for m in block:
        if len(m.srcs) > 0:
            if any([mo.pos == '動詞' for mo in m.morphs]):
                verb = [mo.base for mo in m.morphs if mo.pos == '動詞']
                post = []
                post_term = []
                for src in m.srcs:
                    if any(mo.pos == '助詞' for mo in block[src].morphs):
                        # だけはのような場合は「は」
                        for mo in block[src].morphs:
                            
                        post += [mo.base  if mo.pos == '助詞'][-1]
                        post_term += [''.join([mo.surface for mo in block[src].morphs])]
                if post :
                    post.sort()
                    post_term.sort()
                    case.append(f"{verb[0]}\t{' '.join(post)}\t{' '.join(post_term)}")
print(case[:10])

['生れる\tで\t\u3000どこで', 'つく\tか が\t生れたか 見当が', '泣く\tで\t所で', 'する\tて は\tいた事だけは 泣いて', '始める\tで\tここで', '見る\tは を\tものを 吾輩は', '聞く\tで\tあとで', '捕える\tを\t我々を', '煮る\tて\t捕えて', '食う\tて\t煮て']


### 47. 機能動詞構文のマイニング

In [183]:
# もうちょっと少しきれいにかけそう
pre_path = 'data/chap5/neko_pre.txt'
predicate = []
for block in blockList:
    for m in block:
        if len(m.srcs) > 0:
            # するがある文節
            if any([mo.base == 'する' for mo in m.morphs]):
                #verb = [mo.base for mo in m.morphs if mo.pos == '動詞']
                sahen = ''
                for src in m.srcs:
                    # サ変接続 + を が "する" にかかる
                    if any(block[src].morphs[i].pos1 == 'サ変接続' and block[src].morphs[i+1].base == 'を' for i in range(len(block[src].morphs) -1)):
                        sahen = ''.join([mo.surface for mo in block[src].morphs])
                if sahen :
                    post = []
                    post_term = []
                    # を　を含まない助詞
                    for src in m.srcs:
                        if any(mo.pos == '助詞' for mo in block[src].morphs):
                            post += [mo.base for mo in block[src].morphs if mo.pos == '助詞' and mo.base != 'を']
                            post_term += [''.join([mo.surface for mo in block[src].morphs if mo.pos != '記号'])]
                    if post:
                        post_term = list(filter(lambda x: sahen != x, post_term))
                        post.sort()
                        post_term.sort()
                           
                        predicate.append(f"{sahen + 'する'}\t{' '.join(post)}\t{' '.join(post_term)}")
                        
with open (pre_path, mode='w') as f:
    f.write('\n'.join(predicate))

In [184]:
!cut -f 1  $pre_path | sort | uniq -c | sort -n -r | head

  25 返事をする
  19 挨拶をする
  11 話をする
   8 質問をする
   7 喧嘩をする
   6 真似をする
   5 相談をする
   5 昼寝をする
   4 御辞儀をする
   4 演説をする


### 48. 名詞から根へのパスの抽出

In [207]:
for block in blockList[:10]:
    for m in block:
        dst = int(m.dst)
        if dst > -1:
            if any([mo.pos == '名詞' for mo in m.morphs]):
                start = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
                
                node = []
                while dst != -1:
                    node += [''.join([mo.surface for mo in block[dst].morphs if mo.pos != '記号'])]
                    dst = int(block[dst].dst)
                if node :
                    print(f"{start} -> {' -> '.join(node)}")

吾輩は -> 猫である
名前は -> 無い
どこで -> 生れたか -> つかぬ
見当が -> つかぬ
何でも -> 薄暗い -> 所で -> 泣いて -> 記憶している
所で -> 泣いて -> 記憶している
ニャーニャー -> 泣いて -> 記憶している
いた事だけは -> 記憶している
吾輩は -> 見た
ここで -> 始めて -> 人間という -> ものを -> 見た
人間という -> ものを -> 見た
ものを -> 見た
あとで -> 聞くと -> 種族であったそうだ
それは -> 種族であったそうだ
書生という -> 人間中で -> 種族であったそうだ
人間中で -> 種族であったそうだ
一番 -> 獰悪な -> 種族であったそうだ
獰悪な -> 種族であったそうだ
書生というのは -> 話である
我々を -> 捕えて -> 煮て -> 食うという -> 話である
当時は -> なかったから -> 思わなかった
何という -> 考も -> なかったから -> 思わなかった
考も -> なかったから -> 思わなかった
彼の -> 掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
掌に -> 載せられて -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
スーと -> 持ち上げられた -> 時 -> フワフワした -> 感じが -> あったばかりである
時 -> フワフワした -> 感じが -> あったばかりである
感じが -> あったばかりである


### 49. 名詞間の係り受けパスの抽出

In [230]:
# わからん
for block in blockList[:10]:
    print_sentence(block)
    norn_pair = []
    for i, m in enumerate(block):
        # 名詞があった時
        nornX = ''
        if any([mo.pos == '名詞' for mo in m.morphs]):
            nornX = ''.join([mo.surface for mo in m.morphs if mo.pos != '記号'])
            for i2, m2 in enumerate(block[i+1:]):
                nornY = ''
                if any([mo2.pos == '名詞' for mo2 in m2.morphs]):
                    nornY = ''.join([mo2.surface for mo2 in m2.morphs if mo2.pos != '記号'])
                if nornY:
                    norn_pair.append((nornX, nornY))
    
    # 名詞句pair
        
            
#         # 名詞が文中にある
#         for i, mo in enumerate(m.morphs):
#             if mo.pos == '名詞':
#                 for i2, mo2 in enumerate(m.morphs[i+1:]):
#                     print(mo2.surface)
#                     if mo2.pos == '名詞':
#                         print(mo2.surface)

一
[]
　吾輩は猫である。
[('吾輩は', '猫である')]
名前はまだ無い。
[]
　どこで生れたかとんと見当がつかぬ。
[('どこで', '見当が')]
何でも薄暗いじめじめした所でニャーニャー泣いていた事だけは記憶している。
[('何でも', '所で'), ('何でも', 'ニャーニャー'), ('何でも', 'いた事だけは'), ('何でも', '記憶している'), ('所で', 'ニャーニャー'), ('所で', 'いた事だけは'), ('所で', '記憶している'), ('ニャーニャー', 'いた事だけは'), ('ニャーニャー', '記憶している'), ('いた事だけは', '記憶している')]
吾輩はここで始めて人間というものを見た。
[('吾輩は', 'ここで'), ('吾輩は', '人間という'), ('吾輩は', 'ものを'), ('ここで', '人間という'), ('ここで', 'ものを'), ('人間という', 'ものを')]
しかもあとで聞くとそれは書生という人間中で一番獰悪な種族であったそうだ。
[('あとで', 'それは'), ('あとで', '書生という'), ('あとで', '人間中で'), ('あとで', '一番'), ('あとで', '獰悪な'), ('あとで', '種族であったそうだ'), ('それは', '書生という'), ('それは', '人間中で'), ('それは', '一番'), ('それは', '獰悪な'), ('それは', '種族であったそうだ'), ('書生という', '人間中で'), ('書生という', '一番'), ('書生という', '獰悪な'), ('書生という', '種族であったそうだ'), ('人間中で', '一番'), ('人間中で', '獰悪な'), ('人間中で', '種族であったそうだ'), ('一番', '獰悪な'), ('一番', '種族であったそうだ'), ('獰悪な', '種族であったそうだ')]
この書生というのは時々我々を捕えて煮て食うという話である。
[('書生というのは', '我々を'), ('書生というのは', '話である'), ('我々を', '話である')]
しかしその当時は何という考もなかったから別段恐しいとも思わなかった。
[('当時は', '何という'), (